In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import urllib3
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.action_chains import ActionChains
# import requests 
from urllib import request
from google.colab import files
import sys
from selenium import webdriver

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/FYP/TMF List.csv')

In [ ]:
for index, row in df.iloc[458:460].iterrows():
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    company_name = row['description']
    print(company_name)
    url = row['url']
    print(url)

    if not pd.isna(url):
      driver.get(url)
      wait = WebDriverWait(driver, 20)
      # XPATHS COOKIE MARKETMOVES = [ ' //*[@id="modal−form"]/fieldset/ul[1]/li[1]/label/input ' , ' //*[@id="modal−form"]/fieldset/ul[2]/li[1]/label/input ' , ' //*[@id="gdpr−submit−button"] ' ]

      # for i in range(0, 3):
      #   button = wait.until(EC.element to be clickable ((By.XPATH , XPATHS COOKIE MARKETMOVES[i])))
      #   driver.execute script("arguments[0].click()", button)
      #   time.sleep(3)
      
      try:
        while(driver.find_element_by_xpath('//*[@id="load-more"]').is_displayed()):
          element = driver.find_element_by_xpath('//*[@id="load-more"]')
          driver.execute_script("arguments[0].scrollIntoView();", element)
          time.sleep(3)
          button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="load-more"]')))
          driver.execute_script("arguments[0].click()", button)
          time.sleep(3)

      except:
        pass 

      html = driver.page_source
      f = open('{0}.html'.format(company_name), 'w')
      f.write(html.encode('utf-8').decode('ascii', 'ignore'))
      f.close()
      driver.close()
      http=urllib3.PoolManager()
      data = open('{0}.html'.format(company_name),'r')
      
      soup = BeautifulSoup(data, 'html.parser')
      data1 = soup.find('div', attrs = {'class': 'quote-page-article-listing'})
      links = []
      for link in data1.find_all('a'): 
        links.append(link.get('href'))
      
      links = list(dict.fromkeys(links))
      newlinks = [x for x in links if not x.startswith('https')]
      newlinks = [x for x in newlinks if not x.startswith('/earnings')]

      locals()["df"+str(company_name)] = pd.DataFrame(columns = ['Author', 'Date', 'Headline', 'Text'])


      for link in newlinks: 
        try:
          url = "https://www.fool.com" + link 
          html = request.urlopen(url).read().decode('utf8')
          soup = BeautifulSoup(html, 'html.parser')
          head = soup.find('section', attrs = {'class': 'usmf-new article-header'})
          headline = head.find('h1').string
          sub_headline = head.find('h2').string
          author = soup.find('div', attrs = {'class': 'author-name'}).find('a').string
          dates = soup.find_all('div', attrs = {'class': 'publication-date'})
          dates_list = []
          for date in dates: 
              dates_list.append(date.contents[2].strip())

          for i in dates_list: 
              if i.startswith('Published') and len(dates_list) > 1: 
                  date_final = i
              else: 
                  date_final = i 

          text = soup.find('span', attrs = {'class': 'article-content'})
          paragraphs = text.find_all('p')
          text_final = ""
          for paragraph in paragraphs:
              text_final = text_final + paragraph.getText()

          text_final = text_final.replace('\\', '')
          locals()["df2"+str(company_name)] = pd.DataFrame({"Author":[author], "Date":[date_final], "Headline":[headline], "Text":[text_final]})
          locals()["df"+str(company_name)] = locals()["df"+str(company_name)].append(locals()["df2"+str(company_name)])

        except:
            pass
        
      locals()["df"+str(company_name)].to_csv(r'{0}.csv'.format(company_name))
      files.download("{0}.csv".format(company_name))